In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import scipy.io as sio
import seaborn as sns
%matplotlib inline

In [3]:
data = pd.read_csv('data.csv')
data2 = pd.read_csv('data2.csv')
data = data.set_index('new_time')
data2 = data2.set_index('new_time')
data2.head(1)

,time,sss,sst,wind_spd,wind_dir,precip,cum_precip3,cum_precip6,year,month,...,scaled_sss,scaled_dir,scaled_spd,scaled_sst,pct_change,pct_c_scaled,set,sst_pct_change,dir_pct_change,spd_pct_change
new_time,,,,,,,,,,,,,,,,,,,,,
2016-08-24 02:30:00.000028800,736566.104167,32.785813,29.972728,1.499305,-35.279577,0.0,0.0,0.0,2016,8,...,-0.949913,0.090521,-1.915939,3.406521,0.0,0.0,Spurs2,0.0,0.0,0.0


When we reference these "somewhat" successful models that we've run, they are not always statiscally or predictively successful; rather, they may be models that have, at the least, given us more insight into the problem itself.

# Logistic Classifier

With most problems, a logistic model is a general starting point; we know that with higher dimensions this will not be linear, so it is worth seeing how a logistic model fits.

## Spurs1

We chose to use salinity, wind speed, wind direction, temperature, and the percent change of salinity (all scaled to a normal distribution) for our model. For our gridsearch to optimize parameters, we decided to optimize it towards a recall score, which should reduce the large amount of false positives.

In [14]:
from sklearn.model_selection import train_test_split
X = data.iloc[:, [12,13,14,15,17]] 
y = [1 if i>2.0 else 0 for i in data.iloc[:,6]]
y = pd.DataFrame(y).values
X_train, X_test, y_train, y_test = train_test_split(X, y.ravel(), test_size = 0.25,
                                                    random_state = 0, stratify=y)

In [15]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

classifier = LogisticRegression(solver='lbfgs', max_iter=500, random_state = 0)
classifier.fit(X_train, y_train)
parameters = [{'C': [1, 10, 50, 100], 'multi_class': ['multinomial', 'auto','ovr'],
               'solver': ['lbfgs', 'saga', 'sag','newton-cg'], 'max_iter': [500,2000,5000,10000]}]
grid_search = GridSearchCV(estimator = classifier, param_grid = parameters,
                           scoring = 'recall', cv = 10, n_jobs = 1)
grid_search = grid_search.fit(X_train, y_train.ravel())
best_accuracy = grid_search.best_score_
best_parameters = grid_search.best_params_
print(best_accuracy)
print(best_parameters)
print(grid_search.best_estimator_)

0.04999817491604614
{'C': 10, 'max_iter': 500, 'multi_class': 'multinomial', 'solver': 'lbfgs'}
LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=500, multi_class='multinomial',
          n_jobs=None, penalty='l2', random_state=0, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)


In [18]:
from sklearn.model_selection import cross_val_score
log_class = LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=500, multi_class='multinomial',
          n_jobs=None, penalty='l2', random_state=0, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)
log_class.fit(X_train, y_train)
scores=cross_val_score(log_class, X_test, y_test, cv=4, scoring="recall")
print("scores", scores)
print("Recall: %0.4f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

scores [0.07142857 0.07142857 0.         0.        ]
Recall: 0.0357 (+/- 0.07)


In [19]:
from sklearn.metrics import confusion_matrix
y_pred2 = log_class.predict(X_test)
cm2 = confusion_matrix(y_test, y_pred2)
cm2

array([[2230,    0],
       [  52,    2]], dtype=int64)

So we can see that even at its best, logistic regression cannot tell what causes a rain event

## Spurs2

In [20]:
from sklearn.model_selection import train_test_split
X1 = data2.iloc[:, [12,13,14,15,17]] 
y1 = [1 if i>2.0 else 0 for i in data2.iloc[:,6]]
y1 = pd.DataFrame(y1).values
X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, y1.ravel(), test_size = 0.25,
                                                    random_state = 0, stratify=y1)

In [21]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

classifier = LogisticRegression(solver='lbfgs', max_iter=500, random_state = 0)
classifier.fit(X_train1, y_train1)
parameters = [{'C': [1, 10, 50, 100], 'multi_class': ['multinomial', 'auto','ovr'],
               'solver': ['lbfgs', 'saga', 'sag','newton-cg'], 'max_iter': [500,2000,5000,10000]}]
grid_search = GridSearchCV(estimator = classifier, param_grid = parameters,
                           scoring = 'recall', cv = 10, n_jobs = 1)
grid_search = grid_search.fit(X_train1, y_train1.ravel())
best_accuracy = grid_search.best_score_
best_parameters = grid_search.best_params_
print(best_accuracy)
print(best_parameters)
print(grid_search.best_estimator_)

0.05086779739942892
{'C': 1, 'max_iter': 500, 'multi_class': 'multinomial', 'solver': 'lbfgs'}
LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=500, multi_class='multinomial',
          n_jobs=None, penalty='l2', random_state=0, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)


In [22]:
from sklearn.model_selection import cross_val_score
log_class = LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=500, multi_class='multinomial',
          n_jobs=None, penalty='l2', random_state=0, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)
log_class.fit(X_train1, y_train1)
scores=cross_val_score(log_class, X_test1, y_test1, cv=4, scoring="recall")
print("scores", scores)
print("Recall: %0.4f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

scores [0.04225352 0.02816901 0.02857143 0.01428571]
Recall: 0.0283 (+/- 0.02)


In [23]:
from sklearn.metrics import confusion_matrix
y_pred2 = log_class.predict(X_test1)
cm2 = confusion_matrix(y_test1, y_pred2)
cm2

array([[2327,    6],
       [ 268,   14]], dtype=int64)

# K-Nearest Neighbors

K-Nearest Neighbors is also a good choice for most models because is not so affected by higher dimensions; it takes the Euclidean distance between sets of features, and distance generally translates across dimensions.

## Spurs1

In [24]:
from sklearn.model_selection import train_test_split
X = data.iloc[:, [12,13,14,15,17]] 
y = [1 if i>2.0 else 0 for i in data.iloc[:,6]]
y = pd.DataFrame(y).values
X_train, X_test, y_train, y_test = train_test_split(X, y.ravel(), test_size = 0.25,
                                                    random_state = 0, stratify=y)

In [25]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

knn = KNeighborsClassifier(n_neighbors=5, metric='minkowski')
knn.fit(X_train, y_train)

parameters = [{'n_neighbors': np.arange(1,10), 'algorithm': ['brute', 'auto'], 'p': [1, 2], 'n_jobs': [1]}]
grid_search = GridSearchCV(estimator = knn, param_grid = parameters,
                           scoring = 'recall', cv = 10, n_jobs = 1)
grid_search = grid_search.fit(X_train, y_train)

best_accuracy = grid_search.best_score_
best_parameters = grid_search.best_params_
print(best_accuracy)
print(best_parameters)
print(grid_search.best_estimator_)

0.32500182508395387
{'algorithm': 'brute', 'n_jobs': 1, 'n_neighbors': 1, 'p': 1}
KNeighborsClassifier(algorithm='brute', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=1, p=1,
           weights='uniform')


In [28]:
from sklearn.model_selection import cross_val_score
knn = KNeighborsClassifier(algorithm='brute', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=1, p=1,
           weights='uniform')
knn.fit(X, y.ravel())
scores=cross_val_score(knn, X, y.ravel(), cv=4, scoring="recall")
print("scores", scores)
print("Recall: %0.4f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
y_pred = knn.predict(X_test)
y_pred2 = knn.predict(X_train)
cm = confusion_matrix(y_test, y_pred)
cm2 = confusion_matrix(y_train, y_pred2)
print(cm2)
print()
print(cm)

scores [0.14814815 0.33333333 0.33962264 0.11320755]
Recall: 0.2336 (+/- 0.21)
[[6689    0]
 [   0  160]]

[[2230    0]
 [   0   54]]


In [29]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2230
           1       1.00      1.00      1.00        54

   micro avg       1.00      1.00      1.00      2284
   macro avg       1.00      1.00      1.00      2284
weighted avg       1.00      1.00      1.00      2284



Wow, a perfect score! Rather than a sense of accomplishment, we should be rather concerned about this. What has happened here is that, though we've fit the model on the training set to get our best parameters, when we refit it with more data, KNN has actually "memorized" the data rather than "learning" it. If we were to try and fit the same model on our train data to predict again, the results would be far less pleasing.

In [30]:
knn.fit(X_train, y_train.ravel())
scores=cross_val_score(knn, X_test, y_test.ravel(), cv=4, scoring="recall")
print("scores", scores)
print("Recall: %0.4f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
y_pred = knn.predict(X_test)
y_pred2 = knn.predict(X_train)
cm = confusion_matrix(y_test, y_pred)
cm2 = confusion_matrix(y_train, y_pred2)
print(cm2)
print()
print(cm)

scores [0.21428571 0.21428571 0.07692308 0.38461538]
Recall: 0.2225 (+/- 0.22)
[[6689    0]
 [   0  160]]

[[2209   21]
 [  33   21]]


So again we see a perfect prediction on the train set, but also that KNN now performs very poorly on similar data that it hasn't seen.

## Spurs2

In [31]:
from sklearn.model_selection import train_test_split
X1 = data2.iloc[:, [12,13,14,15,17]] 
y1 = [1 if i>2.0 else 0 for i in data2.iloc[:,6]]
y1 = pd.DataFrame(y1).values
X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, y1.ravel(), test_size = 0.25,
                                                    random_state = 0, stratify=y1)

In [33]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

knn = KNeighborsClassifier(n_neighbors=5, metric='minkowski')
knn.fit(X_train1, y_train1)

parameters = [{'n_neighbors': np.arange(1,10), 'algorithm': ['brute', 'auto'], 'p': [1, 2], 'n_jobs': [1]}]
grid_search = GridSearchCV(estimator = knn, param_grid = parameters,
                           scoring = 'recall', cv = 10, n_jobs = 1)
grid_search = grid_search.fit(X_train1, y_train1)

best_accuracy = grid_search.best_score_
best_parameters = grid_search.best_params_
print(best_accuracy)
print(best_parameters)
print(grid_search.best_estimator_)

0.4366399160378073
{'algorithm': 'brute', 'n_jobs': 1, 'n_neighbors': 1, 'p': 1}
KNeighborsClassifier(algorithm='brute', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=1, p=1,
           weights='uniform')


In [36]:
from sklearn.model_selection import cross_val_score
knn = KNeighborsClassifier(algorithm='brute', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=1, p=1,
           weights='uniform')
knn.fit(X1, y1.ravel())
scores=cross_val_score(knn, X1, y1.ravel(), cv=4, scoring="recall")
print("scores", scores)
print("Recall: %0.4f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
y_pred = knn.predict(X_test1)
y_pred2 = knn.predict(X_train1)
cm = confusion_matrix(y_test1, y_pred)
cm2 = confusion_matrix(y_train1, y_pred2)
print(cm2)
print()
print(cm)

scores [0.44326241 0.32624113 0.18085106 0.30960854]
Recall: 0.3150 (+/- 0.19)
[[6999    0]
 [   0  845]]

[[2333    0]
 [   0  282]]


So this model was useful to us by showing us that even situations this complex were susceptible to overfitting.

# Gradient Boosting

Gradient Boosting is another type of model that has gained quite a bit of traction in the past few years. It is a twist on random forests (clusters of decision trees) built on the idea that combinations of "weak" features in the model can be weighted differently than the strong learners, and "boosted" to fill in a more complete model. It builds off of itself, and is one of the most powerful non-neural network algorithms around.

## XGBoost

### As a Classifier

#### Spurs1

In [38]:
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier

X = data.iloc[:, [12,13,14,15,17]].values 
y = [1 if i>2.0 else 0 for i in data.iloc[:,6]]
y = pd.DataFrame(y).values
X_train, X_test, y_train, y_test = train_test_split(X, y.ravel(), test_size = 0.25,
                                                    random_state = 0, stratify=y)
model = XGBClassifier()
model.fit(X_train, y_train)
pred = model.predict(X_test)
confusion_matrix(y_test, pred)

array([[2221,    9],
       [  39,   15]], dtype=int64)

With gradient boosting models, we can also use a more optimal form of parameter optimization, which actually comes from multiple randomized passes across the hyperplane. This has been shown to be far more effective than discreet (exhaustive) searches in higher dimensions like what we had previously been using.

In [39]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from datetime import datetime
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe

SEED = 314159265
VALID_SIZE = 0.2
TARGET = 'outcome'

def gini(actual, pred, cmpcol = 0, sortcol = 1):
    assert (len(actual) == len(pred))
    all = np.asarray(np.c_[actual, pred, np.arange(len(actual))], dtype=np.float)
    all = all[np.lexsort((all[:, 2], -1 * all[:, 1]))]
    totalLosses = all[:, 0].sum()
    giniSum = all[:, 0].cumsum().sum() / totalLosses

    giniSum -= (len(actual) + 1) / 2.
    return giniSum / len(actual)


def gini_normalized(actual, pred):
    return gini(actual, pred) / gini(actual, actual)

In [78]:
xgb_new = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.75, gamma=0.55, learning_rate=0.25, max_delta_step=0,
       max_depth=9, min_child_weight=5.0, missing=None, n_estimators=348,
       n_jobs=1, nthread=1, objective='binary:logistic', random_state=0,
       reg_alpha=6.0, reg_lambda=1.6, scale_pos_weight=1, seed=None,
       silent=True, subsample=0.75)
xgb_new.fit(X_train, y_train)
pred = xgb_new.predict(X_test)
confusion_matrix(y_test, pred)

array([[2221,    9],
       [  37,   17]], dtype=int64)

In [ ]:
xgb_new.feature_importances_

So even though we still got more false positives, there are far fewer, as well as many more true positives. There will be a certain point where we cannot get any better than the model we have, as many of these factors can be in place and a rain event is still not going to be certain.

#### Spurs2

In [73]:
from sklearn.model_selection import train_test_split
X1 = data2.iloc[:, [12,13,14,15,17]] 
y1 = [1 if i>2.0 else 0 for i in data2.iloc[:,6]]
y1 = pd.DataFrame(y1).values
X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, y1.ravel(), test_size = 0.25,
                                                    random_state = 0, stratify=y1)

In [76]:
xgb_new1 = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.9, gamma=0.6000000000000001, learning_rate=0.25, max_delta_step=0,
       max_depth=12, min_child_weight=1.0, missing=None, n_estimators=278,
       n_jobs=1, nthread=1, objective='binary:logistic', random_state=0,
       reg_alpha=0.0, reg_lambda=1.5, scale_pos_weight=1, seed=None,
       silent=True, subsample=0.75)
xgb_new1.fit(X_train1, y_train1)
pred = xgb_new1.predict(X_test1)
print(xgb_new1.feature_importances_)
confusion_matrix(y_test1, pred)


[0.20892909 0.19477677 0.19550627 0.19725707 0.20353079]


array([[2283,   50],
       [ 194,   88]], dtype=int64)

### As a Regressor

#### Spurs1

In [44]:
from sklearn.model_selection import train_test_split
X = data.iloc[:, [12,13,14,15,17]] 
y = data.iloc[:,6].values
X_train, X_test, y_train, y_test = train_test_split(X, y.ravel(), test_size = 0.25,
                                                    random_state = 0)
from xgboost import XGBRegressor
xgbr = XGBRegressor()
xgbr.fit(X_train.values, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [45]:
import time
import xgboost
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.metrics import mean_absolute_error
import matplotlib.pylab as plt
import pandas as pd, numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error
import xgboost as xgb
from hyperopt import hp, tpe, STATUS_OK, Trials, fmin

#def xgb_eval_mae(yhat, dtrain):
#    y = dtrain.get_label()
 #   return 'mae', mean_absolute_error(np.exp(y), np.exp(yhat))

def objective(space):

    clf = xgb.XGBRegressor(n_estimators = space['n_estimators'],
                           max_depth = space['max_depth'],
                           min_child_weight = space['min_child_weight'],
                           subsample = space['subsample'],
                           learning_rate = space['learning_rate'],
                           gamma = space['gamma'],
                           colsample_bytree = space['colsample_bytree'],
                           objective='reg:linear',
                           scale_pos_weight= space['scale_pos_weight']
                           )

    eval_set  = [( X_train, y_train), ( X_test, y_test)]

    clf.fit(X_train,
            y_train,
            eval_set=eval_set,
            eval_metric = 'rmse')

    pred = clf.predict(X_test)
#   mae = mean_absolute_error(np.exp(y_valid), np.exp(pred))
    mae = mean_absolute_error((y_test), (pred))

#    print "SCORE:", mae
    return{'loss':mae, 'status': STATUS_OK }


space ={
        'max_depth': hp.choice('max_depth', np.arange(10, 30, dtype=int)),
        'min_child_weight': hp.quniform ('min_child', 1, 20, 1),
        'subsample': hp.uniform ('subsample', 0.8, 1),
        'n_estimators' : hp.choice('n_estimators', np.arange(1000, 10000, 100, dtype=int)),
        'learning_rate' : hp.quniform('learning_rate', 0.025, 0.5, 0.025),
        'gamma' : hp.quniform('gamma', 0.5, 1, 0.05),
        'colsample_bytree' : hp.quniform('colsample_bytree', 0.5, 1, 0.05),
        'scale_pos_weight': hp.quniform('scale_pos_weight', 0.5,2.5, 0.25)
    }

trials = Trials()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=3, # change
            trials=trials)

print(best)

[0]	validation_0-rmse:1.53188	validation_1-rmse:1.98547
[1]	validation_0-rmse:1.46832	validation_1-rmse:1.9617
[2]	validation_0-rmse:1.30894	validation_1-rmse:1.82016
[3]	validation_0-rmse:1.23052	validation_1-rmse:1.77265
[4]	validation_0-rmse:1.2012	validation_1-rmse:1.75136
[5]	validation_0-rmse:1.17392	validation_1-rmse:1.72569
[6]	validation_0-rmse:1.15372	validation_1-rmse:1.71777
[7]	validation_0-rmse:1.14556	validation_1-rmse:1.72239
[8]	validation_0-rmse:1.12966	validation_1-rmse:1.7247
[9]	validation_0-rmse:1.06581	validation_1-rmse:1.63873
[10]	validation_0-rmse:1.03683	validation_1-rmse:1.64332
[11]	validation_0-rmse:1.01521	validation_1-rmse:1.63389
[12]	validation_0-rmse:0.990548	validation_1-rmse:1.62609
[13]	validation_0-rmse:0.966528	validation_1-rmse:1.63616
[14]	validation_0-rmse:0.95828	validation_1-rmse:1.63494
[15]	validation_0-rmse:0.94684	validation_1-rmse:1.61311
[16]	validation_0-rmse:0.935144	validation_1-rmse:1.60004
[17]	validation_0-rmse:0.925184	validatio

[142]	validation_0-rmse:0.397593	validation_1-rmse:1.61107
[143]	validation_0-rmse:0.395702	validation_1-rmse:1.61032
[144]	validation_0-rmse:0.393704	validation_1-rmse:1.61361
[145]	validation_0-rmse:0.392239	validation_1-rmse:1.61459
[146]	validation_0-rmse:0.391115	validation_1-rmse:1.6128
[147]	validation_0-rmse:0.390711	validation_1-rmse:1.60736
[148]	validation_0-rmse:0.389687	validation_1-rmse:1.60686
[149]	validation_0-rmse:0.388565	validation_1-rmse:1.60867
[150]	validation_0-rmse:0.38674	validation_1-rmse:1.60501
[151]	validation_0-rmse:0.386863	validation_1-rmse:1.60547
[152]	validation_0-rmse:0.387272	validation_1-rmse:1.60265
[153]	validation_0-rmse:0.38406	validation_1-rmse:1.60434
[154]	validation_0-rmse:0.382729	validation_1-rmse:1.60607
[155]	validation_0-rmse:0.381413	validation_1-rmse:1.60492
[156]	validation_0-rmse:0.37759	validation_1-rmse:1.60581
[157]	validation_0-rmse:0.375434	validation_1-rmse:1.60637
[158]	validation_0-rmse:0.370502	validation_1-rmse:1.60715
[

[282]	validation_0-rmse:0.256936	validation_1-rmse:1.64915
[283]	validation_0-rmse:0.256426	validation_1-rmse:1.64992
[284]	validation_0-rmse:0.256425	validation_1-rmse:1.64993
[285]	validation_0-rmse:0.25642	validation_1-rmse:1.65101
[286]	validation_0-rmse:0.256229	validation_1-rmse:1.65328
[287]	validation_0-rmse:0.255733	validation_1-rmse:1.65138
[288]	validation_0-rmse:0.255353	validation_1-rmse:1.65235
[289]	validation_0-rmse:0.254933	validation_1-rmse:1.65258
[290]	validation_0-rmse:0.253756	validation_1-rmse:1.65165
[291]	validation_0-rmse:0.253653	validation_1-rmse:1.65265
[292]	validation_0-rmse:0.253765	validation_1-rmse:1.65394
[293]	validation_0-rmse:0.252901	validation_1-rmse:1.65233
[294]	validation_0-rmse:0.252333	validation_1-rmse:1.65047
[295]	validation_0-rmse:0.251604	validation_1-rmse:1.6504
[296]	validation_0-rmse:0.251397	validation_1-rmse:1.65081
[297]	validation_0-rmse:0.250791	validation_1-rmse:1.65122
[298]	validation_0-rmse:0.250112	validation_1-rmse:1.65223

[422]	validation_0-rmse:0.225835	validation_1-rmse:1.65756
[423]	validation_0-rmse:0.225835	validation_1-rmse:1.65756
[424]	validation_0-rmse:0.225682	validation_1-rmse:1.65715
[425]	validation_0-rmse:0.22568	validation_1-rmse:1.65714
[426]	validation_0-rmse:0.225679	validation_1-rmse:1.65713
[427]	validation_0-rmse:0.225679	validation_1-rmse:1.65713
[428]	validation_0-rmse:0.225678	validation_1-rmse:1.65712
[429]	validation_0-rmse:0.225679	validation_1-rmse:1.65711
[430]	validation_0-rmse:0.225415	validation_1-rmse:1.65646
[431]	validation_0-rmse:0.225415	validation_1-rmse:1.65646
[432]	validation_0-rmse:0.225217	validation_1-rmse:1.65598
[433]	validation_0-rmse:0.224997	validation_1-rmse:1.65528
[434]	validation_0-rmse:0.224813	validation_1-rmse:1.65638
[435]	validation_0-rmse:0.224815	validation_1-rmse:1.65639
[436]	validation_0-rmse:0.224575	validation_1-rmse:1.65732
[437]	validation_0-rmse:0.224507	validation_1-rmse:1.65599
[438]	validation_0-rmse:0.224352	validation_1-rmse:1.6556

[562]	validation_0-rmse:0.215498	validation_1-rmse:1.65549
[563]	validation_0-rmse:0.215499	validation_1-rmse:1.6555
[564]	validation_0-rmse:0.21554	validation_1-rmse:1.65682
[565]	validation_0-rmse:0.21554	validation_1-rmse:1.65681
[566]	validation_0-rmse:0.215539	validation_1-rmse:1.65681
[567]	validation_0-rmse:0.215474	validation_1-rmse:1.65572
[568]	validation_0-rmse:0.215474	validation_1-rmse:1.65573
[569]	validation_0-rmse:0.215476	validation_1-rmse:1.65574
[570]	validation_0-rmse:0.215477	validation_1-rmse:1.65574
[571]	validation_0-rmse:0.215474	validation_1-rmse:1.65573
[572]	validation_0-rmse:0.215474	validation_1-rmse:1.65571
[573]	validation_0-rmse:0.215517	validation_1-rmse:1.65455
[574]	validation_0-rmse:0.215516	validation_1-rmse:1.65457
[575]	validation_0-rmse:0.215516	validation_1-rmse:1.65457
[576]	validation_0-rmse:0.215517	validation_1-rmse:1.65456
[577]	validation_0-rmse:0.215516	validation_1-rmse:1.65457
[578]	validation_0-rmse:0.215517	validation_1-rmse:1.65457


...

[1030]	validation_0-rmse:0.223	validation_1-rmse:1.63578
[1031]	validation_0-rmse:0.223	validation_1-rmse:1.63578
[1032]	validation_0-rmse:0.223	validation_1-rmse:1.63578
[1033]	validation_0-rmse:0.223	validation_1-rmse:1.63578
[1034]	validation_0-rmse:0.223	validation_1-rmse:1.63578
[1035]	validation_0-rmse:0.223	validation_1-rmse:1.63578
[1036]	validation_0-rmse:0.223	validation_1-rmse:1.63578
[1037]	validation_0-rmse:0.223	validation_1-rmse:1.63578
[1038]	validation_0-rmse:0.223	validation_1-rmse:1.63578
[1039]	validation_0-rmse:0.223	validation_1-rmse:1.63578
[1040]	validation_0-rmse:0.223	validation_1-rmse:1.63578
[1041]	validation_0-rmse:0.223	validation_1-rmse:1.63578
[1042]	validation_0-rmse:0.223	validation_1-rmse:1.63578
[1043]	validation_0-rmse:0.223	validation_1-rmse:1.63578
[1044]	validation_0-rmse:0.223	validation_1-rmse:1.63578
[1045]	validation_0-rmse:0.223	validation_1-rmse:1.63578
[1046]	validation_0-rmse:0.223	validation_1-rmse:1.63578
[1047]	validation_0-rmse:0.223	

[1174]	validation_0-rmse:0.223	validation_1-rmse:1.63578
[1175]	validation_0-rmse:0.223	validation_1-rmse:1.63578
[1176]	validation_0-rmse:0.223	validation_1-rmse:1.63578
[1177]	validation_0-rmse:0.223	validation_1-rmse:1.63578
[1178]	validation_0-rmse:0.223	validation_1-rmse:1.63578
[1179]	validation_0-rmse:0.223	validation_1-rmse:1.63578
[1180]	validation_0-rmse:0.223	validation_1-rmse:1.63578
[1181]	validation_0-rmse:0.223	validation_1-rmse:1.63578
[1182]	validation_0-rmse:0.223	validation_1-rmse:1.63578
[1183]	validation_0-rmse:0.223	validation_1-rmse:1.63578
[1184]	validation_0-rmse:0.223	validation_1-rmse:1.63578
[1185]	validation_0-rmse:0.223	validation_1-rmse:1.63578
[1186]	validation_0-rmse:0.223	validation_1-rmse:1.63578
[1187]	validation_0-rmse:0.223	validation_1-rmse:1.63578
[1188]	validation_0-rmse:0.223	validation_1-rmse:1.63578
[1189]	validation_0-rmse:0.223	validation_1-rmse:1.63578
[1190]	validation_0-rmse:0.223	validation_1-rmse:1.63578
[1191]	validation_0-rmse:0.223	

[1318]	validation_0-rmse:0.223	validation_1-rmse:1.63578
[1319]	validation_0-rmse:0.223	validation_1-rmse:1.63578
[1320]	validation_0-rmse:0.223	validation_1-rmse:1.63578
[1321]	validation_0-rmse:0.223	validation_1-rmse:1.63578
[1322]	validation_0-rmse:0.223	validation_1-rmse:1.63578
[1323]	validation_0-rmse:0.223	validation_1-rmse:1.63578
[1324]	validation_0-rmse:0.223	validation_1-rmse:1.63578
[1325]	validation_0-rmse:0.223	validation_1-rmse:1.63578
[1326]	validation_0-rmse:0.223	validation_1-rmse:1.63578
[1327]	validation_0-rmse:0.223	validation_1-rmse:1.63578
[1328]	validation_0-rmse:0.223	validation_1-rmse:1.63578
[1329]	validation_0-rmse:0.223	validation_1-rmse:1.63578
[1330]	validation_0-rmse:0.223	validation_1-rmse:1.63578
[1331]	validation_0-rmse:0.223	validation_1-rmse:1.63578
[1332]	validation_0-rmse:0.223	validation_1-rmse:1.63578
[1333]	validation_0-rmse:0.223	validation_1-rmse:1.63578
[1334]	validation_0-rmse:0.223	validation_1-rmse:1.63578
[1335]	validation_0-rmse:0.223	

[1457]	validation_0-rmse:0.223	validation_1-rmse:1.63578
[1458]	validation_0-rmse:0.223	validation_1-rmse:1.63578
[1459]	validation_0-rmse:0.223	validation_1-rmse:1.63578
[1460]	validation_0-rmse:0.223	validation_1-rmse:1.63578
[1461]	validation_0-rmse:0.223	validation_1-rmse:1.63578
[1462]	validation_0-rmse:0.223	validation_1-rmse:1.63578
[1463]	validation_0-rmse:0.223	validation_1-rmse:1.63578
[1464]	validation_0-rmse:0.223	validation_1-rmse:1.63578
[1465]	validation_0-rmse:0.223	validation_1-rmse:1.63578
[1466]	validation_0-rmse:0.223	validation_1-rmse:1.63578
[1467]	validation_0-rmse:0.223	validation_1-rmse:1.63578
[1468]	validation_0-rmse:0.223	validation_1-rmse:1.63578
[1469]	validation_0-rmse:0.223	validation_1-rmse:1.63578
[1470]	validation_0-rmse:0.223	validation_1-rmse:1.63578
[1471]	validation_0-rmse:0.223	validation_1-rmse:1.63578
[1472]	validation_0-rmse:0.223	validation_1-rmse:1.63578
[1473]	validation_0-rmse:0.223	validation_1-rmse:1.63578
[1474]	validation_0-rmse:0.223	

[1597]	validation_0-rmse:0.22162	validation_1-rmse:1.63552
[1598]	validation_0-rmse:0.22162	validation_1-rmse:1.63552
[1599]	validation_0-rmse:0.22162	validation_1-rmse:1.63552
{'colsample_bytree': 0.65, 'gamma': 0.7000000000000001, 'learning_rate': 0.375, 'max_depth': 1, 'min_child': 14.0, 'n_estimators': 57, 'scale_pos_weight': 1.0, 'subsample': 0.8224410229685717}


In [47]:
import math
from sklearn.metrics import mean_absolute_error, mean_squared_error
from xgboost import XGBRegressor
xgbr = XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.9500000000000001, gamma=0.7000000000000001, learning_rate=0.30000000000000004,
       max_delta_step=0, max_depth=2, min_child_weight=5, missing=None,
       n_estimators=57, n_jobs=1, nthread=None, objective='reg:linear',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=0.75,
       seed=None, silent=True, subsample=0.9953163392910283)
xgbr.fit(X_train,y_train)
preds = xgbr.predict(X_test)

mae_s1 = mean_absolute_error(y_test, preds)
rmse_s1 = np.sqrt(mean_squared_error(y_test, preds))
print(rmse_s1)
mae_s1

1.5207978872708456


0.340531895462622

#### Spurs2

In [49]:
from sklearn.model_selection import train_test_split
X1 = data2.iloc[:, [12,13,14,15,17]].values
y1 = data2.iloc[:,6].values
X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, y1.ravel(), test_size = 0.25,
                                                    random_state = 0)

In [53]:
import math
from sklearn.metrics import mean_absolute_error, mean_squared_error
from xgboost import XGBRegressor
xgbr = XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.9500000000000001, gamma=0.7000000000000001, learning_rate=0.30000000000000004,
       max_delta_step=0, max_depth=2, min_child_weight=5, missing=None,
       n_estimators=57, n_jobs=1, nthread=None, objective='reg:linear',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=0.75,
       seed=None, silent=True, subsample=0.9953163392910283)
xgbr.fit(X_train1,y_train1)
preds = xgbr.predict(X_test1)

mae_s1 = mean_absolute_error(y_test1, preds)
rmse_s1 = np.sqrt(mean_squared_error(y_test1, preds))
print('RMSE for cumulative precipitation is:', rmse_s1)
print('MAE for cumulative precipitation is:', mae_s1)

RMSE for cumulative precipitation is: 3.677474549265597
MAE for cumulative precipitation is: 1.3421134332430698


In [80]:
xgbr.feature_importances_

array([0.21571648, 0.12942989, 0.2311248 , 0.10785824, 0.31587058],
      dtype=float32)

So using the same exact model as a regressor for both Spurs1 and Spurs2 gives us a relatively acceptable model; we can predict the presence of cumulative precipitation over 3 hours with an average error of .34 mm for Spurs1 and 1.34 for Spurs2. Given the disparity between the MAE and RMSE, however, we can suspect that while the average error is low, there are several very large anomalies in both sets, which we do know is true. We also see how importance change in salinity is, taking up almost a third of the entire model's weight.

## Light GBM

### As a Classifier

#### Spurs1

In [62]:
X = data.iloc[:, [12,13,14,15,17]] 
y = [1 if i>2.0 else 0 for i in data.iloc[:,6]]
y = pd.DataFrame(y).values
X_train, X_test, y_train, y_test = train_test_split(X, y.ravel(), test_size = 0.25,
                                                    random_state = 0, stratify=y)

In [64]:
import lightgbm as lgb
import pandas as pd
from sklearn.metrics import mean_squared_error

estimator = lgb.LGBMClassifier(learning_rate = 0.125, metric = 'l1', 
                        n_estimators = 20, num_leaves = 38)


param_grid = {
    'n_estimators': [x for x in range(20, 36, 2)],
    'learning_rate': [0.10, 0.125, 0.15, 0.175, 0.2]}
gridsearch = GridSearchCV(estimator, param_grid)

gridsearch.fit(X_train, y_train,
        eval_set = [(X_test, y_test)],
        eval_metric = ['auc', 'binary_logloss'],
        early_stopping_rounds = 5)

[1]	valid_0's l1: 0.0454365	valid_0's binary_logloss: 0.089552	valid_0's auc: 0.893506
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's l1: 0.0445239	valid_0's binary_logloss: 0.0840803	valid_0's auc: 0.895595
[3]	valid_0's l1: 0.0434344	valid_0's binary_logloss: 0.0810384	valid_0's auc: 0.881506
[4]	valid_0's l1: 0.0423733	valid_0's binary_logloss: 0.0780939	valid_0's auc: 0.895042
[5]	valid_0's l1: 0.0414893	valid_0's binary_logloss: 0.075689	valid_0's auc: 0.903185
[6]	valid_0's l1: 0.0405365	valid_0's binary_logloss: 0.0740915	valid_0's auc: 0.915977
[7]	valid_0's l1: 0.0395654	valid_0's binary_logloss: 0.0726979	valid_0's auc: 0.916143
[8]	valid_0's l1: 0.0385784	valid_0's binary_logloss: 0.0709831	valid_0's auc: 0.926013
[9]	valid_0's l1: 0.0380326	valid_0's binary_logloss: 0.0704453	valid_0's auc: 0.936435
[10]	valid_0's l1: 0.0375003	valid_0's binary_logloss: 0.0692607	valid_0's auc: 0.937523
[11]	valid_0's l1: 0.036832	valid_0's binary_logloss: 0.0683

[2]	valid_0's l1: 0.0429877	valid_0's binary_logloss: 0.0807374	valid_0's auc: 0.899294
[3]	valid_0's l1: 0.0417511	valid_0's binary_logloss: 0.0772396	valid_0's auc: 0.913096
[4]	valid_0's l1: 0.0403028	valid_0's binary_logloss: 0.074267	valid_0's auc: 0.911805
[5]	valid_0's l1: 0.0391165	valid_0's binary_logloss: 0.0724647	valid_0's auc: 0.910472
[6]	valid_0's l1: 0.038267	valid_0's binary_logloss: 0.0711723	valid_0's auc: 0.910127
[7]	valid_0's l1: 0.0372014	valid_0's binary_logloss: 0.0698262	valid_0's auc: 0.908404
[8]	valid_0's l1: 0.0362787	valid_0's binary_logloss: 0.0682225	valid_0's auc: 0.929821
[9]	valid_0's l1: 0.0354839	valid_0's binary_logloss: 0.0676442	valid_0's auc: 0.927255
[10]	valid_0's l1: 0.0346282	valid_0's binary_logloss: 0.0664645	valid_0's auc: 0.928542
[11]	valid_0's l1: 0.0337239	valid_0's binary_logloss: 0.0651344	valid_0's auc: 0.928621
[12]	valid_0's l1: 0.0330384	valid_0's binary_logloss: 0.064601	valid_0's auc: 0.926395
[13]	valid_0's l1: 0.0323853	val

[1]	valid_0's l1: 0.0449408	valid_0's binary_logloss: 0.0869434	valid_0's auc: 0.900835
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's l1: 0.0438871	valid_0's binary_logloss: 0.08219	valid_0's auc: 0.869449
[3]	valid_0's l1: 0.042722	valid_0's binary_logloss: 0.0795468	valid_0's auc: 0.866318
[4]	valid_0's l1: 0.0416447	valid_0's binary_logloss: 0.0770125	valid_0's auc: 0.866081
[5]	valid_0's l1: 0.0405951	valid_0's binary_logloss: 0.0752027	valid_0's auc: 0.863868
[6]	valid_0's l1: 0.0397888	valid_0's binary_logloss: 0.074279	valid_0's auc: 0.881785
Early stopping, best iteration is:
[1]	valid_0's l1: 0.0449408	valid_0's binary_logloss: 0.0869434	valid_0's auc: 0.900835
[1]	valid_0's l1: 0.0454365	valid_0's binary_logloss: 0.089552	valid_0's auc: 0.893506
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's l1: 0.0445239	valid_0's binary_logloss: 0.0840803	valid_0's auc: 0.895595
[3]	valid_0's l1: 0.0434344	valid_0's binary_logloss: 0.

[17]	valid_0's l1: 0.0335654	valid_0's binary_logloss: 0.0648728	valid_0's auc: 0.949356
[18]	valid_0's l1: 0.0331859	valid_0's binary_logloss: 0.0649135	valid_0's auc: 0.947911
Early stopping, best iteration is:
[13]	valid_0's l1: 0.0356964	valid_0's binary_logloss: 0.0669029	valid_0's auc: 0.949763
[1]	valid_0's l1: 0.0443009	valid_0's binary_logloss: 0.0860765	valid_0's auc: 0.861348
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's l1: 0.0429877	valid_0's binary_logloss: 0.0807374	valid_0's auc: 0.899294
[3]	valid_0's l1: 0.0417511	valid_0's binary_logloss: 0.0772396	valid_0's auc: 0.913096
[4]	valid_0's l1: 0.0403028	valid_0's binary_logloss: 0.074267	valid_0's auc: 0.911805
[5]	valid_0's l1: 0.0391165	valid_0's binary_logloss: 0.0724647	valid_0's auc: 0.910472
[6]	valid_0's l1: 0.038267	valid_0's binary_logloss: 0.0711723	valid_0's auc: 0.910127
[7]	valid_0's l1: 0.0372014	valid_0's binary_logloss: 0.0698262	valid_0's auc: 0.908404
[8]	valid_0's l1: 0.036

[2]	valid_0's l1: 0.0435072	valid_0's binary_logloss: 0.0791842	valid_0's auc: 0.878459
[3]	valid_0's l1: 0.0421894	valid_0's binary_logloss: 0.0771029	valid_0's auc: 0.869503
[4]	valid_0's l1: 0.0411546	valid_0's binary_logloss: 0.0751254	valid_0's auc: 0.883744
[5]	valid_0's l1: 0.0400711	valid_0's binary_logloss: 0.074225	valid_0's auc: 0.91158
[6]	valid_0's l1: 0.0390396	valid_0's binary_logloss: 0.0737769	valid_0's auc: 0.908906
[7]	valid_0's l1: 0.0379866	valid_0's binary_logloss: 0.0729989	valid_0's auc: 0.910899
[8]	valid_0's l1: 0.0370518	valid_0's binary_logloss: 0.0725371	valid_0's auc: 0.91038
[9]	valid_0's l1: 0.0363207	valid_0's binary_logloss: 0.0723156	valid_0's auc: 0.909986
[10]	valid_0's l1: 0.0354685	valid_0's binary_logloss: 0.071764	valid_0's auc: 0.918269
[11]	valid_0's l1: 0.0346344	valid_0's binary_logloss: 0.0715236	valid_0's auc: 0.915928
[12]	valid_0's l1: 0.0338562	valid_0's binary_logloss: 0.0715984	valid_0's auc: 0.925253
[13]	valid_0's l1: 0.0332594	vali

...

[2]	valid_0's l1: 0.0452382	valid_0's binary_logloss: 0.0814265	valid_0's auc: 0.899128
[3]	valid_0's l1: 0.043124	valid_0's binary_logloss: 0.0786082	valid_0's auc: 0.914462
[4]	valid_0's l1: 0.0414198	valid_0's binary_logloss: 0.0769468	valid_0's auc: 0.8988
[5]	valid_0's l1: 0.039298	valid_0's binary_logloss: 0.0730632	valid_0's auc: 0.936593
[6]	valid_0's l1: 0.0380642	valid_0's binary_logloss: 0.0720818	valid_0's auc: 0.94055
[7]	valid_0's l1: 0.0369439	valid_0's binary_logloss: 0.071298	valid_0's auc: 0.942202
[8]	valid_0's l1: 0.0357672	valid_0's binary_logloss: 0.0699678	valid_0's auc: 0.942862
[9]	valid_0's l1: 0.0351546	valid_0's binary_logloss: 0.0703534	valid_0's auc: 0.942651
[10]	valid_0's l1: 0.0342147	valid_0's binary_logloss: 0.0698233	valid_0's auc: 0.947932
[11]	valid_0's l1: 0.033408	valid_0's binary_logloss: 0.0692251	valid_0's auc: 0.950905
[12]	valid_0's l1: 0.0326272	valid_0's binary_logloss: 0.068852	valid_0's auc: 0.950972
[13]	valid_0's l1: 0.031675	valid_0's

[15]	valid_0's l1: 0.0251265	valid_0's binary_logloss: 0.062851	valid_0's auc: 0.940438
[16]	valid_0's l1: 0.0246547	valid_0's binary_logloss: 0.0630372	valid_0's auc: 0.940595
[17]	valid_0's l1: 0.0241946	valid_0's binary_logloss: 0.0631896	valid_0's auc: 0.939603
Early stopping, best iteration is:
[12]	valid_0's l1: 0.0267942	valid_0's binary_logloss: 0.0619305	valid_0's auc: 0.94214
[1]	valid_0's l1: 0.0461451	valid_0's binary_logloss: 0.0812628	valid_0's auc: 0.911381
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's l1: 0.0429219	valid_0's binary_logloss: 0.0775381	valid_0's auc: 0.870864
[3]	valid_0's l1: 0.0404393	valid_0's binary_logloss: 0.0753943	valid_0's auc: 0.885713
[4]	valid_0's l1: 0.0385098	valid_0's binary_logloss: 0.0743171	valid_0's auc: 0.901594
[5]	valid_0's l1: 0.0368493	valid_0's binary_logloss: 0.073377	valid_0's auc: 0.911049
[6]	valid_0's l1: 0.03559	valid_0's binary_logloss: 0.0731508	valid_0's auc: 0.903214
Early stopping, best iter

[1]	valid_0's l1: 0.0429772	valid_0's binary_logloss: 0.0795416	valid_0's auc: 0.866517
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's l1: 0.0403289	valid_0's binary_logloss: 0.0755736	valid_0's auc: 0.914375
[3]	valid_0's l1: 0.0379721	valid_0's binary_logloss: 0.0726585	valid_0's auc: 0.92681
[4]	valid_0's l1: 0.0361247	valid_0's binary_logloss: 0.0704217	valid_0's auc: 0.932021
[5]	valid_0's l1: 0.0343667	valid_0's binary_logloss: 0.0687521	valid_0's auc: 0.930938
[6]	valid_0's l1: 0.032749	valid_0's binary_logloss: 0.0676619	valid_0's auc: 0.916397
[7]	valid_0's l1: 0.0314144	valid_0's binary_logloss: 0.0656297	valid_0's auc: 0.925241
[8]	valid_0's l1: 0.0303869	valid_0's binary_logloss: 0.0645252	valid_0's auc: 0.933504
[9]	valid_0's l1: 0.0293864	valid_0's binary_logloss: 0.0635823	valid_0's auc: 0.937054
[10]	valid_0's l1: 0.0284216	valid_0's binary_logloss: 0.0628217	valid_0's auc: 0.937897
[11]	valid_0's l1: 0.027355	valid_0's binary_logloss: 0.0620

[2]	valid_0's l1: 0.0414027	valid_0's binary_logloss: 0.0793102	valid_0's auc: 0.892256
[3]	valid_0's l1: 0.0394729	valid_0's binary_logloss: 0.0767044	valid_0's auc: 0.903222
[4]	valid_0's l1: 0.0379176	valid_0's binary_logloss: 0.0755143	valid_0's auc: 0.899967
[5]	valid_0's l1: 0.0364757	valid_0's binary_logloss: 0.0737612	valid_0's auc: 0.903853
[6]	valid_0's l1: 0.0351435	valid_0's binary_logloss: 0.0727526	valid_0's auc: 0.917734
[7]	valid_0's l1: 0.0340536	valid_0's binary_logloss: 0.0716929	valid_0's auc: 0.918174
[8]	valid_0's l1: 0.0329217	valid_0's binary_logloss: 0.0698059	valid_0's auc: 0.927632
[9]	valid_0's l1: 0.0320662	valid_0's binary_logloss: 0.0687542	valid_0's auc: 0.941418
[10]	valid_0's l1: 0.0312577	valid_0's binary_logloss: 0.0682881	valid_0's auc: 0.94109
[11]	valid_0's l1: 0.0307504	valid_0's binary_logloss: 0.0685856	valid_0's auc: 0.938731
[12]	valid_0's l1: 0.0299843	valid_0's binary_logloss: 0.066962	valid_0's auc: 0.953097
[13]	valid_0's l1: 0.0293639	va

GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        importance_type='split', learning_rate=0.125, max_depth=-1,
        metric='l1', min_child_samples=20, min_child_weight=0.001,
        min_split_gain=0.0, n_estimators=20, n_jobs=-1, num_leaves=38,
        objective=None, random_state=None, reg_alpha=0.0, reg_lambda=0.0,
        silent=True, subsample=1.0, subsample_for_bin=200000,
        subsample_freq=0),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'n_estimators': [20, 22, 24, 26, 28, 30, 32, 34], 'learning_rate': [0.1, 0.125, 0.15, 0.175, 0.2]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [65]:
print('Best parameters found by grid search are:', gridsearch.best_params_)

Best parameters found by grid search are: {'learning_rate': 0.175, 'n_estimators': 20}


In [66]:
gbm = lgb.LGBMClassifier(learning_rate = 0.125, metric = 'l1', 
                        n_estimators = 20)
gbm.fit(X_train, y_train,
        eval_set=[(X_test, y_test)],
        eval_metric=['auc', 'binary_logloss'],
early_stopping_rounds=5)

[1]	valid_0's l1: 0.0442044	valid_0's binary_logloss: 0.0819194	valid_0's auc: 0.847774
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's l1: 0.042562	valid_0's binary_logloss: 0.0785732	valid_0's auc: 0.884583
[3]	valid_0's l1: 0.041145	valid_0's binary_logloss: 0.0757202	valid_0's auc: 0.899332
[4]	valid_0's l1: 0.0397871	valid_0's binary_logloss: 0.0739667	valid_0's auc: 0.902566
[5]	valid_0's l1: 0.0387755	valid_0's binary_logloss: 0.0726195	valid_0's auc: 0.900739
[6]	valid_0's l1: 0.0376291	valid_0's binary_logloss: 0.0708754	valid_0's auc: 0.923912
[7]	valid_0's l1: 0.0365857	valid_0's binary_logloss: 0.0695402	valid_0's auc: 0.930232
[8]	valid_0's l1: 0.0356721	valid_0's binary_logloss: 0.0684951	valid_0's auc: 0.930377
[9]	valid_0's l1: 0.0348878	valid_0's binary_logloss: 0.067798	valid_0's auc: 0.931029
[10]	valid_0's l1: 0.0341852	valid_0's binary_logloss: 0.067098	valid_0's auc: 0.931785
[11]	valid_0's l1: 0.0335734	valid_0's binary_logloss: 0.06684

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        importance_type='split', learning_rate=0.125, max_depth=-1,
        metric='l1', min_child_samples=20, min_child_weight=0.001,
        min_split_gain=0.0, n_estimators=20, n_jobs=-1, num_leaves=31,
        objective=None, random_state=None, reg_alpha=0.0, reg_lambda=0.0,
        silent=True, subsample=1.0, subsample_for_bin=200000,
        subsample_freq=0)

In [67]:
from sklearn import metrics
y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration_)
print(metrics.confusion_matrix(y_test, y_pred))

[[2218   12]
 [  38   16]]


#### Spurs2

In [68]:
from sklearn.model_selection import train_test_split
X1 = data2.iloc[:, [12,13,14,15,17]] 
y1 = [1 if i>2.0 else 0 for i in data2.iloc[:,6]]
y1 = pd.DataFrame(y1).values
X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, y1.ravel(), test_size = 0.25,
                                                    random_state = 0, stratify=y1)

In [71]:
gbm = lgb.LGBMClassifier(learning_rate = 0.15, metric = 'l1', 
                        n_estimators = 20)
gbm.fit(X_train1, y_train1,
        eval_set=[(X_test1, y_test1)],
        eval_metric=['auc', 'binary_logloss'],
early_stopping_rounds=5)

[1]	valid_0's l1: 0.186089	valid_0's binary_logloss: 0.313111	valid_0's auc: 0.818454
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's l1: 0.180658	valid_0's binary_logloss: 0.297161	valid_0's auc: 0.835588
[3]	valid_0's l1: 0.175845	valid_0's binary_logloss: 0.287034	valid_0's auc: 0.841689
[4]	valid_0's l1: 0.171949	valid_0's binary_logloss: 0.280106	valid_0's auc: 0.843793
[5]	valid_0's l1: 0.168567	valid_0's binary_logloss: 0.275014	valid_0's auc: 0.845254
[6]	valid_0's l1: 0.165412	valid_0's binary_logloss: 0.270858	valid_0's auc: 0.846021
[7]	valid_0's l1: 0.162706	valid_0's binary_logloss: 0.266265	valid_0's auc: 0.850982
[8]	valid_0's l1: 0.160124	valid_0's binary_logloss: 0.262691	valid_0's auc: 0.853774
[9]	valid_0's l1: 0.157835	valid_0's binary_logloss: 0.26032	valid_0's auc: 0.853434
[10]	valid_0's l1: 0.15602	valid_0's binary_logloss: 0.258231	valid_0's auc: 0.854431
[11]	valid_0's l1: 0.154043	valid_0's binary_logloss: 0.256198	valid_0's auc: 0.

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        importance_type='split', learning_rate=0.15, max_depth=-1,
        metric='l1', min_child_samples=20, min_child_weight=0.001,
        min_split_gain=0.0, n_estimators=20, n_jobs=-1, num_leaves=31,
        objective=None, random_state=None, reg_alpha=0.0, reg_lambda=0.0,
        silent=True, subsample=1.0, subsample_for_bin=200000,
        subsample_freq=0)

In [72]:
from sklearn import metrics
y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration_)
print(metrics.confusion_matrix(y_test, y_pred))

[[2209   21]
 [  43   11]]


### As a Regressor

#### Spurs1

In [ ]:
from sklearn.model_selection import train_test_split
X = data.iloc[:, [12,13,14,15,17]] 
y = data.iloc[:,6].values
X_train, X_test, y_train, y_test = train_test_split(X, y.ravel(), test_size = 0.25, random_state = 0)

In [54]:
import lightgbm as lgb
import pandas as pd
from sklearn.metrics import mean_squared_error

# create dataset for lightgbm
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

# specify your configurations as a dict
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2', 'l1'},
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}

print('Starting training...')
# train
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=100,
                valid_sets=lgb_eval,
                early_stopping_rounds=5)

print('Saving model...')
# save model to file
gbm.save_model('model.txt')

print('Starting predicting...')
# predict
y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)
# eval
print('The rmse of prediction is:', mean_squared_error(y_test, y_pred) ** 0.5)
print('The mae of prediction is:', mean_absolute_error(y_test, y_pred))

Starting training...
[1]	valid_0's l1: 0.438008	valid_0's l2: 4.36073
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's l1: 0.430262	valid_0's l2: 4.25839
[3]	valid_0's l1: 0.422828	valid_0's l2: 4.16143
[4]	valid_0's l1: 0.416128	valid_0's l2: 4.07393
[5]	valid_0's l1: 0.410552	valid_0's l2: 3.99823
[6]	valid_0's l1: 0.404711	valid_0's l2: 3.92465
[7]	valid_0's l1: 0.39916	valid_0's l2: 3.85758
[8]	valid_0's l1: 0.39509	valid_0's l2: 3.80607
[9]	valid_0's l1: 0.390633	valid_0's l2: 3.74913
[10]	valid_0's l1: 0.387151	valid_0's l2: 3.70373
[11]	valid_0's l1: 0.383773	valid_0's l2: 3.65636
[12]	valid_0's l1: 0.380103	valid_0's l2: 3.61462
[13]	valid_0's l1: 0.376904	valid_0's l2: 3.57485
[14]	valid_0's l1: 0.373807	valid_0's l2: 3.52942
[15]	valid_0's l1: 0.370669	valid_0's l2: 3.49292
[16]	valid_0's l1: 0.36785	valid_0's l2: 3.44877
[17]	valid_0's l1: 0.365255	valid_0's l2: 3.40819
[18]	valid_0's l1: 0.363272	valid_0's l2: 3.37152
[19]	valid_0's l1: 0.361228	va

So this model actually got even smaller error, down to an average error of 0.33 mm over a span of 3 hours. One other interesting thing to note is that LightGBM did its own parameter optimization, and tested itself within 1 second, which is about 70X faster than XGBoost.

#### Spurs2

In [ ]:
from sklearn.model_selection import train_test_split
X1 = data2.iloc[:, [12,13,14,15,17]] 
y1 = data2.iloc[:,6].values
X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, y1.ravel(), test_size = 0.25,
                                                    random_state = 0, stratify=y1)

In [56]:
import lightgbm as lgb
import pandas as pd
from sklearn.metrics import mean_squared_error

# create dataset for lightgbm
lgb_train = lgb.Dataset(X_train1, y_train1)
lgb_eval = lgb.Dataset(X_test1, y_test1, reference=lgb_train)

# specify your configurations as a dict
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2', 'l1'},
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}

print('Starting training...')
# train
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=100,
                valid_sets=lgb_eval,
                early_stopping_rounds=5)

print('Saving model...')
# save model to file
gbm.save_model('model.txt')

print('Starting predicting...')
# predict
y_pred = gbm.predict(X_test1, num_iteration=gbm.best_iteration)
# eval
print('The rmse of prediction is:', mean_squared_error(y_test1, y_pred) ** 0.5)
print('The mae of prediction is:', mean_absolute_error(y_test1, y_pred))

Starting training...
[1]	valid_0's l1: 1.81992	valid_0's l2: 18.3401
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's l1: 1.79078	valid_0's l2: 17.9281
[3]	valid_0's l1: 1.76235	valid_0's l2: 17.5274
[4]	valid_0's l1: 1.73502	valid_0's l2: 17.1436
[5]	valid_0's l1: 1.71038	valid_0's l2: 16.8328
[6]	valid_0's l1: 1.6893	valid_0's l2: 16.5687
[7]	valid_0's l1: 1.66814	valid_0's l2: 16.3255
[8]	valid_0's l1: 1.64988	valid_0's l2: 16.1085
[9]	valid_0's l1: 1.63297	valid_0's l2: 15.9283
[10]	valid_0's l1: 1.61837	valid_0's l2: 15.7571
[11]	valid_0's l1: 1.60029	valid_0's l2: 15.5091
[12]	valid_0's l1: 1.58303	valid_0's l2: 15.275
[13]	valid_0's l1: 1.56703	valid_0's l2: 15.0695
[14]	valid_0's l1: 1.55326	valid_0's l2: 14.8942
[15]	valid_0's l1: 1.54033	valid_0's l2: 14.7764
[16]	valid_0's l1: 1.52645	valid_0's l2: 14.6566
[17]	valid_0's l1: 1.51441	valid_0's l2: 14.5876
[18]	valid_0's l1: 1.50067	valid_0's l2: 14.4791
[19]	valid_0's l1: 1.49087	valid_0's l2: 14.360

So again, we see that we get lower error compared to the XGBoost model, and this is still running 4-5 times faster than that model. Also, looking at the distribution of Spurs2 and its cumulative precipitation, we see that each standard deviation is so large that an average error of 1.33 mm is actually relatively small. It is not nearly what we were hoping to get, but it is far better than any other model.

In [79]:
gbm.feature_importances_

array([137, 110, 112, 140, 101])

In [57]:
data2['cum_precip3'].describe()

count    1.045900e+04
mean     1.126457e+00
std      4.386322e+00
min     -7.993606e-15
25%      0.000000e+00
50%      1.776357e-15
75%      6.622220e-02
max      9.143455e+01
Name: cum_precip3, dtype: float64

# Conclusions

On the surface, it seems that none of these models came close to being successful. Every confusion matrix was plagued with false positives, and though our regressors seemed to perform very well, they do not predict any abnormalities. However, each of these models was "successful" in its own way: logistic, for instance, gave us insight into just how complex this model would be, by being so unsuccessful. K-Nearest Neighbors, on the other hand, told us that though this model was complex, it was still not immune to overfitting. Gradient Boosting, while the classifiers still had difficulties parsing out the false positives, confirmed the presence of large outliers in each set with the large difference between MAE and RMSE. Furthermore, it told us that change in salinity does indeed play a large role in classifying rain events.

## So What Next?

Now that we have gleaned these insights from these models, the first thing that we'd like to do is see if we can review what we classify as "rain events". For now, we classify rain events as 2 millimeters of precipitation over the course of 3 hours. However, time clearly plays a role in the change in ocean salinity, and to properly look at this effect we must look at change by hour and the responsive rain (or lack thereof). We want to see if there is either a general class we can create, or if we can find a way to classify rain events for each respective climate that our buoys are in.